# LLM Tags - Полный пример с Ollama

Этот notebook демонстрирует полный цикл работы с библиотекой `llm_tags` для автоматического тегирования обращений клиентов.

## Этапы работы:
1. **Первичная обработка всех данных** - тегирование всех обращений из demo_sample.csv.gz
2. **Дополнительный анализ не тегнутых обращений** - повторная обработка обращений без тегов
3. **Финальная переразметка всех обращений** - полная переразметка с учетом накопленных знаний

## Требования:
- Ollama должна быть запущена: `ollama serve`
- Модель должна быть установлена: `ollama pull qwen2.5:32b`


## Подготовка


In [1]:
# Импорт библиотек
import pandas as pd
import sys
from pathlib import Path

sys.path.insert(0, '/home/alex/tradeML/llm_tags')

from llm_tags import TaggingPipeline, OllamaLLM

print("✓ Библиотеки загружены!")


✓ Библиотеки загружены!


In [2]:
# Загрузка данных из demo_sample.csv.gz
data_path = "/home/alex/tradeML/llm_tags/demo_sample.csv.gz"

df = pd.read_csv(data_path, compression='gzip')

print(f"Загружено {len(df)} обращений")
print(f"Колонки: {list(df.columns)}")
print(f"\nПервые 5 строк:")
df.head()


Загружено 1000 обращений
Колонки: ['conversation_id', 'speaker', 'date_time', 'text', 'request_id']

Первые 5 строк:


,conversation_id,speaker,date_time,text,request_id
0,c32e822d23da45c2bb85184905226d75,agent,2023-09-07T08:12:21.384618+00:00,(to herself) That was a bit of a challenging c...,demo-0000
1,6d28ea9a0c094c4187f255c0b2cde753,agent,2023-10-05T15:18:32.384615+00:00,"Great, thank you! Now, can you please confirm ...",demo-0001
2,7b987c77f3524fbd817b90e8b9d167c3,agent,2023-11-27T11:03:30.384615+00:00,"Don't worry, Isabella. We'll take care of that...",demo-0002
3,ce9e10ae96b04a35a6bce60ae6f06e81,client,2023-09-15T11:59:01+00:00,"Hi Tressie, thanks for picking up my call agai...",demo-0003
4,dc37cb3f16864b23945ae4e973332c63,agent,2023-11-01T13:41:46.461541+00:00,"You're welcome, Katheryn. Just to recize, I've...",demo-0004


In [3]:
# Статистика по данным
print("Статистика по данным:")
print(f"  Всего обращений: {len(df)}")
print(f"  Уникальных conversation_id: {df['conversation_id'].nunique()}")
print(f"  Уникальных request_id: {df['request_id'].nunique()}")
print(f"\nРаспределение по speaker:")
print(df['speaker'].value_counts())
print(f"\nПримеры текстов:")
for i, text in enumerate(df['text'].head(3), 1):
    print(f"  {i}. {text[:100]}...")


Статистика по данным:
  Всего обращений: 1000
  Уникальных conversation_id: 481
  Уникальных request_id: 1000

Распределение по speaker:
speaker
agent     529
client    471
Name: count, dtype: int64

Примеры текстов:
  1. (to herself) That was a bit of a challenging call, but I'm glad I could help Felix resolve his issue...
  2. Great, thank you! Now, can you please confirm your name and phone number for with the account?...
  3. Don't worry, Isabella. We'll take care of that for you. I'm going to add a note to your account so w...


In [4]:
# Инициализация LLM и Pipeline
llm = OllamaLLM(
    api_url="http://localhost:11434/api",
    model="qwen3:32b",
    temperature=0.7
)

pipeline = TaggingPipeline(
    llm=llm,
    batch_size=30,  # Обрабатываем по 50 обращений за раз
    num_workers=5   # 5 параллельных потоков
)

print("✓ LLM и Pipeline инициализированы")
print(f"  Модель: {llm.model}")
print(f"  Batch size: {pipeline.batch_size}")
print(f"  Workers: {pipeline.num_workers}")


✓ LLM и Pipeline инициализированы
  Модель: qwen3:32b
  Batch size: 30
  Workers: 5


In [ ]:
# Промпт для тегирования
tag_prompt = """
Проанализируй обращения клиентов телеком-оператора Union Mobile и определи теги для каждого.
Теги должны отражать основную проблему обращения.

"""

print("✓ Промпт подготовлен")


✓ Промпт подготовлен


## Этап 1: Первичная обработка всех данных

На этом этапе мы обрабатываем все обращения из файла и создаем первичную разметку.


In [6]:
print("=" * 80)
print("ЭТАП 1: ПЕРВИЧНАЯ ОБРАБОТКА ВСЕХ ДАННЫХ")
print("=" * 80)

# Запускаем тегирование всех обращений
# skip_if_tags_count=999 означает обработать все строки (у которых < 999 тегов, т.е. все)
result_df_stage1, tags_dict_stage1 = pipeline.tag(
    tags=df,
    text_column="text",
    tag_prompt=tag_prompt,
    id_column="request_id",
    skip_if_tags_count=999,  # Обработать все строки (обрабатываются строки где count < skip_if_tags_count)
    max_tags=5
)

print("\n✓ Этап 1 завершен!")


ЭТАП 1: ПЕРВИЧНАЯ ОБРАБОТКА ВСЕХ ДАННЫХ
Всего обращений: 1000
Для обработки: 1000
Пропущено (уже есть 999+ тегов): 0
Батчей для обработки: 34


Обработка батчей (параллельно):   0%|          | 0/34 [00:00<?, ?batch/s]

Обработка батчей (параллельно):   6%|▌         | 2/34 [00:11<03:26,  6.45s/batch, завершено=3, тегов=0]

[ERROR] Failed to parse JSON: Expecting ',' delimiter: line 1 column 200 (char 199)
[ERROR] Full content: {"tags": ["Security", "DeviceLock"], "descriptions": {"Security": "Client is providing a security code for a device lock.", "DeviceLock": "Agent is initiating a remote lock for the client's device."}
[ERROR] Attempt 1 failed: Expecting ',' delimiter: line 1 column 201 (char 200)
[SUCCESS] Attempt 2 succeeded! Parsed 1 objects


Обработка батчей (параллельно): 100%|██████████| 34/34 [02:47<00:00,  4.94s/batch, завершено=34, тегов=8]

Обработка завершена. Всего тегов в словаре: 8

✓ Этап 1 завершен!


In [7]:
result_df_stage1[result_df_stage1['tags'] != '']

,conversation_id,speaker,date_time,text,request_id,tags
90,37a54558e77c4e3bbfdb6c0aacc8a054,agent,2023-09-01T11:27:21.846153+00:00,"Okay, let's take a closer look. Have you check...",demo-0090,"Support, DataUsage"
120,12bf1224d4da4489bbb5313333fc1b69,agent,2023-09-18T12:09:44+00:00,"Hello, thank you for calling Union Mobile's 24...",demo-0120,"greeting, customer_service"
150,23db1c8756ef42b380898ddac2f332ca,client,2023-09-01T10:05:08.307692+00:00,"Hi Annetta, I'm calling because I've lost my p...",demo-0150,"Утрата телефона, Удаленное блокирование/стирание"
180,2b026228bba547819010297cde33101d,agent,2023-10-19T09:48:43.923076+00:00,"Great! Now, I'd like to test the connection to...",demo-0180,"network_settings, troubleshooting"
270,c0779d855e3f446ebe21d7d90ce61961,agent,2023-12-10T15:28:16.923077+00:00,Thank you for going through those troubleshoot...,demo-0270,customer_service
330,fcd5163c08b84ab69ec426bf6a4119fe,agent,2023-12-01T13:48:22.846155+00:00,"Great! Now that we phone is in safe mode, let'...",demo-0330,Support
360,d6c152860ecc47a9b95263f63ea6da92,client,2023-12-23T17:28:04.769231+00:00,"Hi Kaitlyn, thanks for picking up my call. I'v...",demo-0360,Support
450,179128b0382748e2acb9aac6b40fb852,client,2023-11-02T12:55:37.538461+00:00,"It charge was on January 15th, and it's for $2...",demo-0450,customer_service
510,d074873ae98b47d8ac5e1faabf9d7cb0,agent,2023-09-08T08:14:39.461539+00:00,"Great! Install ahead and install the update, a...",demo-0510,Support
660,0953e97a547c4db78205343303d0e7aa,agent,2023-09-08T16:53:59.384617+00:00,"Thank you for holding, Alexandra. I've checked...",demo-0660,"Support, network_settings"


In [8]:
#tags_dict_stage1

In [18]:
# Анализ результатов этапа 1
print("Результаты этапа 1:")
print(f"  Всего обращений: {len(result_df_stage1)}")
print(f"  Всего уникальных тегов: {len(tags_dict_stage1)}")

# Подсчет обращений с тегами и без
has_tags = result_df_stage1['tags'].apply(lambda x: bool(x and str(x).strip()))
print(f"  Обращений с тегами: {has_tags.sum()}")
print(f"  Обращений без тегов: {(~has_tags).sum()}")

# Топ-10 самых популярных тегов
all_tags = []
for tags_str in result_df_stage1['tags']:
    if tags_str and str(tags_str).strip():
        all_tags.extend([t.strip() for t in str(tags_str).split(',')])

from collections import Counter
tag_counts = Counter(all_tags)
print(f"\nТоп-10 самых популярных тегов:")
for tag, count in tag_counts.most_common(10):
    print(f"  {tag}: {count}")


Результаты этапа 1:
  Всего обращений: 1000
  Всего уникальных тегов: 0
  Обращений с тегами: 0
  Обращений без тегов: 1000

Топ-10 самых популярных тегов:


In [19]:
# Примеры размеченных обращений
print("Примеры размеченных обращений:")
print("\n" + "="*80)
for i, row in result_df_stage1.head(10).iterrows():
    print(f"ID: {row['request_id']}")
    print(f"Speaker: {row['speaker']}")
    print(f"Text: {row['text'][:150]}...")
    print(f"Tags: {row['tags']}")
    print("="*80)


Примеры размеченных обращений:

ID: demo-0000
Speaker: agent
Text: (to herself) That was a bit of a challenging call, but I'm glad I could help Felix resolve his issue. I hope he found a better experience with us in t...
Tags: 
ID: demo-0001
Speaker: agent
Text: Great, thank you! Now, can you please confirm your name and phone number for with the account?...
Tags: 
ID: demo-0002
Speaker: agent
Text: Don't worry, Isabella. We'll take care of that for you. I'm going to add a note to your account so waive any data charges incurred while you're troubl...
Tags: 
ID: demo-0003
Speaker: client
Text: Hi Tressie, thanks for picking up my call again. I'm having trouble with my data connection. It keeps dropping every few minutes, and I'm getting frus...
Tags: 
ID: demo-0004
Speaker: agent
Text: You're welcome, Katheryn. Just to recize, I've updated your plan to the 5GB Plan and changed your billing address. Is there anything else I can assist...
Tags: 
ID: demo-0005
Speaker: client
Text: Wow, th

In [20]:
# Словарь тегов с описаниями
print(f"Словарь тегов ({len(tags_dict_stage1)} тегов):")
print("\n" + "="*80)
for tag, description in sorted(tags_dict_stage1.items()):
    print(f"• {tag}")
    if description:
        print(f"  {description}")
    print()


Словарь тегов (0 тегов):



In [21]:
# Сохраняем результаты этапа 1
output_file_stage1 = "/home/alex/tradeML/llm_tags/results_stage1.csv"
result_df_stage1.to_csv(output_file_stage1, index=False)
print(f"✓ Результаты этапа 1 сохранены в: {output_file_stage1}")

# Сохраняем словарь тегов
tags_file_stage1 = "/home/alex/tradeML/llm_tags/tags_dict_stage1.json"
import json
with open(tags_file_stage1, 'w', encoding='utf-8') as f:
    json.dump(tags_dict_stage1, f, ensure_ascii=False, indent=2)
print(f"✓ Словарь тегов сохранен в: {tags_file_stage1}")


✓ Результаты этапа 1 сохранены в: /home/alex/tradeML/llm_tags/results_stage1.csv
✓ Словарь тегов сохранен в: /home/alex/tradeML/llm_tags/tags_dict_stage1.json


## Этап 2: Дополнительный анализ не тегнутых обращений

На этом этапе мы анализируем обращения, которые не получили теги на первом этапе, и пытаемся их дотегировать.


In [11]:
print("=" * 80)
print("ЭТАП 2: ДОПОЛНИТЕЛЬНЫЙ АНАЛИЗ НЕ ТЕГНУТЫХ ОБРАЩЕНИЙ")
print("=" * 80)

# Фильтруем обращения без тегов
no_tags_mask = result_df_stage1['tags'].apply(lambda x: not (x and str(x).strip()))
no_tags_df = result_df_stage1[no_tags_mask].copy()

print(f"\nНайдено обращений без тегов: {len(no_tags_df)}")

if len(no_tags_df) > 0:
    print("\nПримеры обращений без тегов:")
    for i, row in no_tags_df.head(5).iterrows():
        print(f"  {row['request_id']}: {row['text'][:100]}...")


ЭТАП 2: ДОПОЛНИТЕЛЬНЫЙ АНАЛИЗ НЕ ТЕГНУТЫХ ОБРАЩЕНИЙ

Найдено обращений без тегов: 1000

Примеры обращений без тегов:
  demo-0000: (to herself) That was a bit of a challenging call, but I'm glad I could help Felix resolve his issue...
  demo-0001: Great, thank you! Now, can you please confirm your name and phone number for with the account?...
  demo-0002: Don't worry, Isabella. We'll take care of that for you. I'm going to add a note to your account so w...
  demo-0003: Hi Tressie, thanks for picking up my call again. I'm having trouble with my data connection. It keep...
  demo-0004: You're welcome, Katheryn. Just to recize, I've updated your plan to the 5GB Plan and changed your bi...


In [ ]:
# Создаем более детальный промпт для сложных случаев
detailed_prompt = """
Проанализируй обращения клиентов телеком-оператора Union Mobile.
Эти обращения не удалось классифицировать ранее, поэтому будь особенно внимателен.

Определи теги для каждого обращения, даже если контекст неясен:
- Если это часть диалога, попытайся понять тему из контекста
- Если это приветствие/прощание агента, используй тег "обслуживание"
- Если это короткий ответ клиента ("да", "нет", "спасибо"), используй тег "диалог"
- Если упоминается конкретная проблема, используй соответствующий тег

Категории тегов:
- авторизация, пароль, тарифы, биллинг, технические_проблемы
- роуминг, настройки, устройство, sim_карта, доставка
- жалоба, консультация, обслуживание, диалог

Создавай новые теги если нужно.
"""

if len(no_tags_df) > 0:
    # Повторная обработка не тегнутых обращений
    result_df_stage2, tags_dict_stage2 = pipeline.tag(
        tags=result_df_stage1.copy(),  # Используем результаты этапа 1
        text_column="text",
        tag_prompt=detailed_prompt,
        id_column="request_id",
        existing_tags=tags_dict_stage1,  # Передаем существующие теги
        skip_if_tags_count=1,  # Обработать только строки без тегов (0 тегов)
        max_tags=3
    )
    
    print("\n✓ Этап 2 завершен!")
else:
    print("\n✓ Все обращения уже имеют теги, этап 2 пропущен")
    result_df_stage2 = result_df_stage1.copy()
    tags_dict_stage2 = tags_dict_stage1.copy()


Всего обращений: 1000
Для обработки: 1000
Пропущено (уже есть 1+ тегов): 0
Батчей для обработки: 20


In [ ]:
# Анализ результатов этапа 2
print("Результаты этапа 2:")
print(f"  Всего обращений: {len(result_df_stage2)}")
print(f"  Всего уникальных тегов: {len(tags_dict_stage2)}")
print(f"  Новых тегов добавлено: {len(tags_dict_stage2) - len(tags_dict_stage1)}")

# Подсчет обращений с тегами и без
has_tags_stage2 = result_df_stage2['tags'].apply(lambda x: bool(x and str(x).strip()))
print(f"  Обращений с тегами: {has_tags_stage2.sum()}")
print(f"  Обращений без тегов: {(~has_tags_stage2).sum()}")
print(f"  Улучшение: +{has_tags_stage2.sum() - has_tags.sum()} обращений получили теги")


In [ ]:
# Сравнение этапов 1 и 2
print("Сравнение этапов 1 и 2:")
print(f"\nОбращения, которые получили теги на этапе 2:")

# Находим обращения, которые не имели тегов на этапе 1, но получили на этапе 2
newly_tagged = result_df_stage2[
    (~result_df_stage1['tags'].apply(lambda x: bool(x and str(x).strip()))) &
    (result_df_stage2['tags'].apply(lambda x: bool(x and str(x).strip())))
]

print(f"Всего: {len(newly_tagged)} обращений\n")
for i, row in newly_tagged.head(10).iterrows():
    print(f"ID: {row['request_id']}")
    print(f"Text: {row['text'][:100]}...")
    print(f"New Tags: {row['tags']}")
    print("="*80)


In [ ]:
# Сохраняем результаты этапа 2
output_file_stage2 = "/home/alex/tradeML/llm_tags/results_stage2.csv"
result_df_stage2.to_csv(output_file_stage2, index=False)
print(f"✓ Результаты этапа 2 сохранены в: {output_file_stage2}")

# Сохраняем обновленный словарь тегов
tags_file_stage2 = "/home/alex/tradeML/llm_tags/tags_dict_stage2.json"
with open(tags_file_stage2, 'w', encoding='utf-8') as f:
    json.dump(tags_dict_stage2, f, ensure_ascii=False, indent=2)
print(f"✓ Словарь тегов сохранен в: {tags_file_stage2}")


## Этап 3: Финальная переразметка всех обращений

На этом этапе мы делаем финальную переразметку всех обращений с учетом накопленных знаний и словаря тегов.


In [ ]:
print("=" * 80)
print("ЭТАП 3: ФИНАЛЬНАЯ ПЕРЕРАЗМЕТКА ВСЕХ ОБРАЩЕНИЙ")
print("=" * 80)

# Финальный промпт с учетом всех накопленных знаний
final_prompt = """
Проанализируй обращения клиентов телеком-оператора Union Mobile и определи финальные теги.
Используй существующие теги из словаря, они были созданы на основе анализа всех обращений.

Правила:
1. Приоритет - использование существующих тегов из словаря
2. Создавай новые теги только если существующие не подходят
3. Используй от 1 до 3 наиболее релевантных тегов
4. Будь консистентным - похожие обращения должны иметь похожие теги

Контекст:
- Это финальная разметка после двух этапов анализа
- Словарь тегов уже содержит проверенные категории
- Цель - создать качественную и консистентную разметку
"""

print("\nЗапуск финальной переразметки...")
print(f"Используется словарь из {len(tags_dict_stage2)} тегов")


In [ ]:
# Финальная переразметка всех обращений
result_df_final, tags_dict_final = pipeline.tag(
    tags=df.copy(),  # Начинаем с исходных данных
    text_column="text",
    tag_prompt=final_prompt,
    id_column="request_id",
    existing_tags=tags_dict_stage2,  # Используем накопленный словарь
    skip_if_tags_count=0,  # Переразметить все обращения
    max_tags=3
)

print("\n✓ Этап 3 завершен!")


In [ ]:
# Финальная статистика
print("=" * 80)
print("ФИНАЛЬНЫЕ РЕЗУЛЬТАТЫ")
print("=" * 80)

print(f"\nОбщая статистика:")
print(f"  Всего обращений: {len(result_df_final)}")
print(f"  Всего уникальных тегов: {len(tags_dict_final)}")

# Подсчет обращений с тегами
has_tags_final = result_df_final['tags'].apply(lambda x: bool(x and str(x).strip()))
print(f"  Обращений с тегами: {has_tags_final.sum()} ({has_tags_final.sum()/len(result_df_final)*100:.1f}%)")
print(f"  Обращений без тегов: {(~has_tags_final).sum()} ({(~has_tags_final).sum()/len(result_df_final)*100:.1f}%)")

# Статистика по количеству тегов
tags_count = result_df_final['tags'].apply(
    lambda x: len([t.strip() for t in str(x).split(',') if t.strip()]) if x and str(x).strip() else 0
)
print(f"\nРаспределение по количеству тегов:")
print(f"  0 тегов: {(tags_count == 0).sum()}")
print(f"  1 тег: {(tags_count == 1).sum()}")
print(f"  2 тега: {(tags_count == 2).sum()}")
print(f"  3 тега: {(tags_count == 3).sum()}")
print(f"  Среднее количество тегов: {tags_count.mean():.2f}")


In [ ]:
# Топ-20 самых популярных тегов
all_tags_final = []
for tags_str in result_df_final['tags']:
    if tags_str and str(tags_str).strip():
        all_tags_final.extend([t.strip() for t in str(tags_str).split(',')])

tag_counts_final = Counter(all_tags_final)
print(f"\nТоп-20 самых популярных тегов:")
for i, (tag, count) in enumerate(tag_counts_final.most_common(20), 1):
    print(f"  {i:2d}. {tag:30s} : {count:5d} ({count/len(result_df_final)*100:5.2f}%)")


In [ ]:
# Примеры финальной разметки
print("\nПримеры финальной разметки:")
print("\n" + "="*80)

# Показываем разнообразные примеры
sample_indices = result_df_final.sample(min(15, len(result_df_final))).index
for i in sample_indices:
    row = result_df_final.loc[i]
    print(f"ID: {row['request_id']}")
    print(f"Speaker: {row['speaker']}")
    print(f"Text: {row['text'][:150]}...")
    print(f"Tags: {row['tags']}")
    print("="*80)


In [ ]:
# Финальный словарь тегов
print(f"\nФинальный словарь тегов ({len(tags_dict_final)} тегов):")
print("\n" + "="*80)
for tag, description in sorted(tags_dict_final.items()):
    count = tag_counts_final.get(tag, 0)
    print(f"• {tag} (использован {count} раз)")
    if description:
        print(f"  {description}")
    print()


In [ ]:
# Сравнение всех этапов
print("\n" + "="*80)
print("СРАВНЕНИЕ ВСЕХ ЭТАПОВ")
print("="*80)

comparison_data = {
    'Метрика': [
        'Всего обращений',
        'Обращений с тегами',
        'Обращений без тегов',
        'Уникальных тегов',
        'Среднее тегов на обращение'
    ],
    'Этап 1': [
        len(result_df_stage1),
        has_tags.sum(),
        (~has_tags).sum(),
        len(tags_dict_stage1),
        result_df_stage1['tags'].apply(
            lambda x: len([t.strip() for t in str(x).split(',') if t.strip()]) if x and str(x).strip() else 0
        ).mean()
    ],
    'Этап 2': [
        len(result_df_stage2),
        has_tags_stage2.sum(),
        (~has_tags_stage2).sum(),
        len(tags_dict_stage2),
        result_df_stage2['tags'].apply(
            lambda x: len([t.strip() for t in str(x).split(',') if t.strip()]) if x and str(x).strip() else 0
        ).mean()
    ],
    'Этап 3 (Финал)': [
        len(result_df_final),
        has_tags_final.sum(),
        (~has_tags_final).sum(),
        len(tags_dict_final),
        tags_count.mean()
    ]
}

comparison_df = pd.DataFrame(comparison_data)
comparison_df


In [ ]:
# Сохраняем финальные результаты
output_file_final = "/home/alex/tradeML/llm_tags/results_final.csv"
result_df_final.to_csv(output_file_final, index=False)
print(f"✓ Финальные результаты сохранены в: {output_file_final}")

# Сохраняем финальный словарь тегов
tags_file_final = "/home/alex/tradeML/llm_tags/tags_dict_final.json"
with open(tags_file_final, 'w', encoding='utf-8') as f:
    json.dump(tags_dict_final, f, ensure_ascii=False, indent=2)
print(f"✓ Финальный словарь тегов сохранен в: {tags_file_final}")

# Сохраняем сравнительную таблицу
comparison_file = "/home/alex/tradeML/llm_tags/comparison.csv"
comparison_df.to_csv(comparison_file, index=False)
print(f"✓ Сравнительная таблица сохранена в: {comparison_file}")


## Анализ качества разметки


In [ ]:
# Анализ по типу speaker
print("Статистика по типу speaker:")
print("\n" + "="*80)

for speaker in result_df_final['speaker'].unique():
    speaker_df = result_df_final[result_df_final['speaker'] == speaker]
    has_tags_speaker = speaker_df['tags'].apply(lambda x: bool(x and str(x).strip()))
    
    print(f"\n{speaker.upper()}:")
    print(f"  Всего обращений: {len(speaker_df)}")
    print(f"  С тегами: {has_tags_speaker.sum()} ({has_tags_speaker.sum()/len(speaker_df)*100:.1f}%)")
    print(f"  Без тегов: {(~has_tags_speaker).sum()} ({(~has_tags_speaker).sum()/len(speaker_df)*100:.1f}%)")
    
    # Топ-5 тегов для этого speaker
    speaker_tags = []
    for tags_str in speaker_df['tags']:
        if tags_str and str(tags_str).strip():
            speaker_tags.extend([t.strip() for t in str(tags_str).split(',')])
    
    if speaker_tags:
        speaker_tag_counts = Counter(speaker_tags)
        print(f"  Топ-5 тегов:")
        for tag, count in speaker_tag_counts.most_common(5):
            print(f"    - {tag}: {count}")


In [ ]:
# Анализ обращений без тегов (если есть)
untagged_df = result_df_final[~has_tags_final]

if len(untagged_df) > 0:
    print(f"\nОбращения без тегов ({len(untagged_df)}):")
    print("\n" + "="*80)
    
    for i, row in untagged_df.head(20).iterrows():
        print(f"ID: {row['request_id']}")
        print(f"Speaker: {row['speaker']}")
        print(f"Text: {row['text']}")
        print("="*80)
else:
    print("\n✓ Все обращения успешно размечены!")


## Итоги

Мы успешно прошли все три этапа:

1. **Этап 1**: Первичная обработка всех данных - создали базовую разметку
2. **Этап 2**: Дополнительный анализ не тегнутых обращений - улучшили покрытие
3. **Этап 3**: Финальная переразметка всех обращений - создали консистентную разметку

Все результаты сохранены в файлы:
- `results_stage1.csv` - результаты этапа 1
- `results_stage2.csv` - результаты этапа 2
- `results_final.csv` - финальные результаты
- `tags_dict_final.json` - финальный словарь тегов
- `comparison.csv` - сравнительная таблица
